# IA Practica 3: Introducción a PyTorch
## Hecho por: Juan Mario Sosa Romo 320051926
### v0.1 23/03/25

### Imports y planteamiento del automata


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# Entrada: [estado_bit1, estado_bit2, entrada]
X = torch.tensor([
    [0, 0, 0],  # Estado 00, entrada 0 -> salida [0,1]
    [0, 0, 1],  # Estado 00, entrada 1 -> salida [0,0]
    [0, 1, 0],  # Estado 01, entrada 0 -> salida [1,0]
    [0, 1, 1],  # Estado 01, entrada 1 -> salida [0,0]
    [1, 0, 0],  # Estado 10, entrada 0 -> salida [1,1]
    [1, 0, 1],  # Estado 10, entrada 1 -> salida [1,0]
    [1, 1, 0],  # Estado 11, entrada 0 -> salida [0,0]
    [1, 1, 1]   # Estado 11, entrada 1 -> salida [1,0]
], dtype=torch.float32)

# Salida
Y = torch.tensor([
    [0, 1],
    [0, 0],
    [1, 0],
    [0, 0],
    [1, 1],
    [1, 0],
    [0, 0],
    [1, 0]
], dtype=torch.float32)

### Creando el modelo

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Transforma 3 entradas a 2 salidas
        self.fc = nn.Linear(3, 2)
        # Función sigmoide para la activación
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc(x)
        x = self.sigmoid(x)
        return x

In [ ]:
# Función de pérdida: Binary Cross Entropy (BCE)
criterion = nn.BCELoss()

# Optimizador: Stochastic Gradient Descent (SGD)
learning_rate = 0.1
model = SimpleNet()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Best : .05 con .2812 con 10000000 epochs (1 hora xd)
# Best : 1.5 con .2812 cualquier cosa encima tambien lleva a esto

In [ ]:
# Número de épocas de entr enamiento
num_epochs = 1000000

for epoch in range(num_epochs):
    # Forward propagation
    outputs = model(X)

    # Calcular la discrepancia
    loss = criterion(outputs, Y)

    # Backward propagation
    optimizer.zero_grad()  # Limpia los gradientes del optimizador
    loss.backward()        # Calcula los gradientes (backpropagation)
    optimizer.step()       # Actualiza los pesos usando el optimizador

    # Imprimir la pérdida cada 100 épocas para ver la evolución
    if (epoch+1) % 100000 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [100000/1000000], Loss: 0.2821
Epoch [200000/1000000], Loss: 0.2816
Epoch [300000/1000000], Loss: 0.2815
Epoch [400000/1000000], Loss: 0.2814
Epoch [500000/1000000], Loss: 0.2813
Epoch [600000/1000000], Loss: 0.2813
Epoch [700000/1000000], Loss: 0.2813
Epoch [800000/1000000], Loss: 0.2813
Epoch [900000/1000000], Loss: 0.2813
Epoch [1000000/1000000], Loss: 0.2813


### Evaluacion del modelo

In [ ]:
# Evaluar el modelo con los mismos datos de entrenamiento
with torch.no_grad():  # No necesitamos calcular gradientes en esta fase
    predictions = model(X)
    print("Predicciones:")
    print(predictions)


Predicciones:
tensor([[2.5001e-01, 9.9954e-01],
        [2.5001e-01, 1.5159e-04],
        [2.5001e-01, 1.5159e-04],
        [2.5000e-01, 1.0689e-11],
        [7.5000e-01, 9.9963e-01],
        [7.5000e-01, 1.8831e-04],
        [7.5000e-01, 1.8831e-04],
        [7.4999e-01, 1.3279e-11]])


#### Aca hago una vista mas bonita para ver diferencia entre bits (esperados vs reales)

In [ ]:
pred_labels = (predictions >= 0.5).float()

correct_count = 0
total_bits = 0

print("Comparación de cada bit (verde = correcto, rojo = incorrecto):\n")

for i in range(pred_labels.shape[0]):
    row_str = ""
    for j in range(pred_labels.shape[1]):
        total_bits += 1
        pred_bit = int(pred_labels[i][j].item())
        true_bit = int(Y[i][j].item())
        if pred_bit == true_bit:
            row_str += f"\033[92m{pred_bit}\033[0m "  # Verde
            correct_count += 1
        else:
            row_str += f"\033[91m{pred_bit}\033[0m "  # Rojo
    print(row_str)

# Contadores finales
incorrect_count = total_bits - correct_count
print("\nContadores:")
print(f"Bits correctos: {correct_count} de {total_bits}")
print(f"Bits incorrectos: {incorrect_count} de {total_bits}")


Comparación de cada bit (verde = correcto, rojo = incorrecto):

0 1 
0 0 
0 0 
0 0 
1 1 
1 0 
1 0 
1 0 

Contadores:
Bits correctos: 14 de 16
Bits incorrectos: 2 de 16
